In [ ]:
import pandas as pd
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Solar Flare Classifier

The goal of this analysis is to write a Support Vector Machine which will classify solar flare type. Our output class will roughly be the "flare_class" column of the DataFrame below. For simplicity, I have boiled the flare class to simply be a letter. "B" for all B class flares, "C", for all C class flares, "A" for all A class flares, "M" for all M class flares, and "X" for all X class flares. In the file `xray_data_wrangling.ipynb`, we have code which one-hot-encodes "flare_class" column. These are the columns "B", "C", "M", "X", and "A". The input features will be the "xrsb_flux", the "background_flux" and the "integrated_flux", all of which will be standardized.

Note: To eliminate noise, we will consider only the PEAK x-ray flux data of each solar event. This means that we will consider only the rows of the DataFrame which have a "status" of "EVENT_PEAK". 

In [ ]:
xrayData = pd.read_csv("data/xrsSummaryOneHotEncoded.csv")
xrayData.dropna(inplace=True)
xrayData["flux_standardized"] = (xrayData['xrsb_flux'] - xrayData['xrsb_flux'].mean()) / xrayData['xrsb_flux'].std()
xrayData["background_standardized"] = (xrayData['background_flux'] - xrayData['background_flux'].mean()) / xrayData['background_flux'].std()
xrayData["integrated_standardized"] = (xrayData['integrated_flux'] - xrayData['integrated_flux'].mean()) / xrayData['integrated_flux'].std()
display(xrayData.head())
# Consider only the peak of each solar event
xrayData = xrayData[xrayData.status == "EVENT_PEAK"]
xrayData.head()

## Define Input Features and Output Classes

First, drop all "A" class flares. There are only **two**, so if both are includinig in training or if both are included in testing an error will be thrown

In [ ]:
xrayData = xrayData[xrayData['flare_type'] != "A"]

split = 0.7
trainCount = int(split * len(xrayData))
trainIndices = np.random.choice(xrayData.index,trainCount,replace=False)
train = xrayData[xrayData.index.isin(trainIndices)]
test = xrayData[xrayData.index.isin(trainIndices) == False]
# train = xrayData[0:cutoff]
# test = xrayData[cutoff:-1]
features = ['flux_standardized', 'background_standardized', 'integrated_standardized']
# features = ['xrsb_flux', 'background_flux', 'integrated_flux']
X = xrayData[features]
trainX = train[features]
testX = test[features]

# classNames = ["A","B", "C", "M", "X"]
classNames = ["B", "C", "M", "X"]
classes = ['flare_type']
y = xrayData[classes]
trainY = train[classes]
testY = test[classes]


## Plot Data with Classes

In [ ]:
size = len(xrayData)
subset = xrayData[0:size]
step = [i for i in range(size)]

In [ ]:
sns.scatterplot(data=subset, x=step, y='xrsb_flux', hue='flare_type', marker=".")
plt.yscale("log")
plt.title("XRS-B Flux Over Time with Flare Type")
plt.xlabel("Time Step")
plt.show()
sns.scatterplot(data=subset, x=step, y='background_flux', hue='flare_type', marker=".")
plt.yscale("log")
plt.xlabel("Time Step")
plt.title("Background Flux Over Time with Flare Type")
plt.show()
sns.scatterplot(data=subset, x=step, y='integrated_flux', hue='flare_type', marker=".")
plt.yscale("log")
plt.title("Integrated Flux Over Time with Flare Type")
plt.xlabel("Time Step")
plt.show()

## Define ML Model:

In [ ]:
classifier = svm.SVC(kernel='rbf', class_weight={"B": 200, "C": 200, "M": 5, "X": 100})
classifier.fit(trainX,trainY.values.reshape(len(trainY),))

In [ ]:
y_pred = classifier.predict(testX)
test = test.assign(prediction=y_pred)
metrics = {"precision": 0, "recall": 0, "f-score": 0, "support": 0}
p, r, f, s = precision_recall_fscore_support(testY, y_pred, labels=classNames)
metrics['precision'] = p
metrics["recall"] = r
metrics['f-score'] = f
metrics['support'] = s
metrics


In [ ]:
disp = ConfusionMatrixDisplay.from_estimator(classifier, testX, testY, display_labels=classNames)
disp.ax_.set_title("Confusion Matrix")

## Plot Predictions vs. Actual Results

In [ ]:
test['correct'] = test['flare_type'] == test['prediction']
sns.scatterplot(data=test, x=[i for i in range(len(test))], y="xrsb_flux", hue='correct', marker='.')
plt.yscale("log")
plt.title("Graph of XRS-B Flux and if the SVM Predicted Correctly")
plt.show()

In [ ]:
# Number of false predictions
display(len(test[test.correct == False]))

In [ ]:
def MonteCarloCrossValidation(n, split, data, features, classes, classNames):
    clf = svm.SVC(kernel='rbf', class_weight={"B": 200, "C": 200, "M": 5, "X": 100})
    metrics = {}
    for cls in classNames:
        metrics[cls] = {"p": 0, "r": 0, "f": 0, "s": 0}
    for _ in range(n):
        trainCount = int(split * len(data))
        trainIndices = np.random.choice(data.index,trainCount,replace=False)
        train = data[data.index.isin(trainIndices)]
        test = data[data.index.isin(trainIndices) == False]
        trainX = train[features]
        trainY = train[classes]
        testX = test[features]
        testY = test[classes]

        clf.fit(trainX,trainY.values.reshape(len(trainY),))
        predY = clf.predict(testX)
        p, r, f, s = precision_recall_fscore_support(testY, predY)
        for i in range(len(classNames)):
            metrics[classNames[i]]["p"] += p[i]
            metrics[classNames[i]]["r"] += r[i]
            metrics[classNames[i]]["f"] += f[i]
            metrics[classNames[i]]["s"] += s[i]
    for val in metrics.values():
        for key in val.keys():
            val[key] /= n
    return metrics

display("Average p,r,f,s for each class:")
MonteCarloCrossValidation(10,0.7,xrayData,features,classes,classNames)

## Now, try predicting based on only one feature

We use Monte Carlo Cross Validation to see how our model does with only one feature

In [ ]:
display("Based on Standardized Flux")
display(MonteCarloCrossValidation(4,0.7,xrayData,["flux_standardized"], classes, classNames))
display("Based on Standardized Background Flux")
display(MonteCarloCrossValidation(4,0.7,xrayData,["background_standardized"], classes, classNames))
display("Based on Standardized Integrated Flux")
MonteCarloCrossValidation(4,0.7,xrayData,["integrated_standardized"], classes, classNames)

**Result:**  Model seems to perform *better* with only the standardized XRS-B flux. Let's compare that model with the full model, and with a model excluding the background flux, which seems to perform the worst.

In [ ]:
display(MonteCarloCrossValidation(10,0.7,xrayData,['flux_standardized'],classes, classNames))
display(MonteCarloCrossValidation(10,0.7,xrayData,['flux_standardized','integrated_standardized'],classes, classNames))
MonteCarloCrossValidation(10,0.7,xrayData,features,classes, classNames)

The model performs best when only the standardized XRS-B flux is used, so we will simplify our model:

In [ ]:
# -- DEFINE INPUT AND OUTPUT --
features = ['flux_standardized']
# features = ['xrsb_flux', 'background_flux', 'integrated_flux']
X = xrayData[features]
trainX = train[features]
testX = test[features]

# classNames = ["A","B", "C", "M", "X"]
classNames = ["B", "C", "M", "X"]
classes = ['flare_type']
y = xrayData[classes]
trainY = train[classes]
testY = test[classes]

# -- DEFINE MODEL --
classifier = svm.SVC(kernel='rbf', class_weight={"B": 200, "C": 200, "M": 5, "X": 100})
classifier.fit(trainX,trainY.values.reshape(len(trainY),))

# -- PREDICT --
y_pred = classifier.predict(testX)
test = test.assign(prediction=y_pred)
metrics = {"precision": 0, "recall": 0, "f-score": 0, "support": 0}
p, r, f, s = precision_recall_fscore_support(testY, y_pred, labels=classNames)
metrics['precision'] = p
metrics["recall"] = r
metrics['f-score'] = f
metrics['support'] = s
display(metrics)

# -- MAKE PLOTS --
disp = ConfusionMatrixDisplay.from_estimator(classifier, testX, testY, display_labels=classNames)
disp.ax_.set_title("Confusion Matrix")
plt.show()

test['correct'] = test['flare_type'] == test['prediction']
sns.scatterplot(data=test, x=[i for i in range(len(test))], y="xrsb_flux", hue='correct', marker='.')
plt.yscale("log")
plt.title("Graph of XRS-B Flux and if the SVM Predicted Correctly")
plt.show()

fig, ax = plt.subplots(1,2, figsize=(10,5))
display(ax)
sns.scatterplot(data=test, x=[i for i in range(len(test))], y="xrsb_flux", hue='flare_type', marker='.', ax=ax[0])
ax[0].set_title("XRS-B Flux with Actual Flare Class")
ax[0].set_yscale('log')
ax[0].set_xlabel("Point Label")
sns.scatterplot(data=test, x=[i for i in range(len(test))], y="xrsb_flux", hue='prediction', marker='.', ax=ax[1])
ax[1].set_title("XRS-B Flux with Predicted Flare Class")
ax[1].set_yscale('log')
ax[1].set_xlabel("Point Label")
plt.show()

# Cross validation
display(MonteCarloCrossValidation(20,0.7,xrayData,['flux_standardized'],classes, classNames))

# Number of false predictions
display(len(test[test.correct == False]))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,5))
display(ax)
sns.scatterplot(data=test, x=[i for i in range(len(test))], y="xrsb_flux", hue='flare_type', marker='.', ax=ax[0])
ax[0].set_title("XRS-B Flux with Actual Flare Class")
ax[0].set_yscale('log')
ax[0].set_xlabel("Point Label")
sns.scatterplot(data=test, x=[i for i in range(len(test))], y="xrsb_flux", hue='prediction', marker='.', ax=ax[1])
ax[1].set_title("XRS-B Flux with Predicted Flare Class")
ax[1].set_yscale('log')
ax[1].set_xlabel("Point Label")
plt.show()

len(train) + len(test)